In [1]:
import os

if os.getcwd().split("/")[-2] == "notebooks":
    os.chdir("../../")

import os
import httpx
import xml.etree.ElementTree as ET
from typing import TypedDict, Optional

from dotenv import load_dotenv
load_dotenv("env/.env.yandex_search")

True

In [19]:
async def fetch_search_results(base_link, folder_id, api_key, query):
    url = f"{base_link}?folderid={folder_id}&apikey={api_key}&query={query}"
    async with httpx.AsyncClient() as client:
        response = await client.get(url)
        response.raise_for_status()
        return response.text

def parse_xml_response(xml_data):
    root = ET.fromstring(xml_data)
    results = []
    for group in root.findall(".//group"):
        doc = group.find(".//doc")
        if doc is not None:
            url = doc.find(".//url").text if doc.find(".//url") is not None else "No URL"
            domain = doc.find(".//domain").text if doc.find(".//domain") is not None else "No domain"
            title_element = doc.find(".//title")
            title = "".join(title_element.itertext()).strip() if title_element is not None else "No title"
            results.append({"url": url, "domain": domain, "title": title})
    return results

class ProductInfo(TypedDict):
    brand_name: str
    model_name: str
    part_number: Optional[str]

In [26]:
product_info = ProductInfo(brand_name="ACER", model_name="CC715-91P-X7V8", part_number="NX.C5FER.001")

product_info

{'brand_name': 'ACER',
 'model_name': 'CC715-91P-X7V8',
 'part_number': 'NX.C5FER.001'}

In [27]:
base_link = os.getenv("YANDEX_SEARCH_BASE_LINK")
folder_id = os.getenv("YANDEX_SEARCH_FOLDER_ID")
api_key = os.getenv("YANDEX_SEARCH_API_KEY")

query = f"{product_info['brand_name']} {product_info['model_name']} {product_info['part_number']}"
response = await fetch_search_results(base_link, folder_id, api_key, query)
parse_xml_response(response)

[{'url': 'https://www.acer.com/ru-ru/conceptd/laptops/conceptd-7-ezel/pdp/NX.C5FER.001',
  'domain': 'www.acer.com',
  'title': 'ConceptD 7 Ezel - CC715-91P-X7V8 Технические... | Acer Россия'},
 {'url': 'https://ac-russia.com.ru/noutbuk-conceptd-7-ezel-pro-cc715-91p-x7v8-nxc5fer001/',
  'domain': 'ac-russia.com.ru',
  'title': 'ACER Ноутбук ConceptD 7 Ezel Pro... | Купить ac-russia.com.ru'},
 {'url': 'https://market.yandex.ru/product--noutbuk-acer-conceptd-7-ezel-pro-cc715-91p-x7v8-3840x2160-intel-xeon-2-4-ggts-ram-32-gb-ssd-2-tb-quadro-rtx-5000-win10-pro/668440018?sku=100955322731',
  'domain': 'market.yandex.ru',
  'title': '15.6" Ноутбук Acer ConceptD 7 Ezel Pro... | Яндекс Маркет'},
 {'url': 'https://www.citilink.ru/product/noutbuk-acer-conceptd-7-ezel-cc715-91p-x7v8-i7-10885m-32gb-ssd1tb-1tb-1383001/properties/',
  'domain': 'www.citilink.ru',
  'title': 'Характеристики Ноутбук игровой Acer ConceptD 7 Ezel...'},
 {'url': 'https://www.citilink.ru/product/noutbuk-acer-conceptd-7-eze